# Morpheme Finder with Random Forest
We will do morpheme segmentation with random forest in this notebook.
## Import & Constants Definition

In [ ]:
from json import loads
from requests import request, ConnectionError
from sklearn import ensemble
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, cross_val_score
from tqdm.notebook import tqdm

In [ ]:
# env variables
try:
    with open('.env.json') as f:
        ENV_VARIABLES = loads(f.read())
        f.close()
except FileNotFoundError:
    ENV_VARIABLES = {'DATA_DIR': 'C:\\'}
DATA_DIR = ENV_VARIABLES['DATA_DIR']
FTP_DIR = 'http://m106.nthu.edu.tw/~s106062341/morpheme_finder_data/'

# file accessor
def get_file(filename: str, callback: classmethod) -> bool:
    try:
        with open(f'{DATA_DIR}{filename}', 'r') as f:
            callback(f.read())
            f.close()
            return True
    except FileNotFoundError:
        try:
            res = request('GET', f'{FTP_DIR}{filename}')
            res.encoding = 'Big5'
            callback(res.text)
            return True
        except ConnectionError:
            print('HTTP connection failed')
            return False
        except Exception as e:
            print(f'Load failed: {e}')
            return False

class Word:
    def __init__(self, text, origin_affix_list, affix_list=None) -> None:
        self.text = text
        self.origin_affix_list = origin_affix_list
        self.affix_list = affix_list if affix_list else origin_affix_list
        self.label = None
        self.create_label()
        
    def create_label(self) -> None:
        text = self.text
        label = [0] * len(text)
        pos = 0
        for affix in self.affix_list[1:]:
            prev_pos = text.find(affix, pos)
            label[prev_pos] = 1
            pos = prev_pos + len(affix)
        self.label = label
        
    def get_breakpoints_index(self) -> list:
        return [position for position, label in enumerate(self.label) if label]

## Load Data & Create its Label
1. CELEX.word.and.root.txt

In [ ]:
word_dict = {}
bad_celex = []

def celex_word_and_root_callback(content: str) -> any:
    for line in tqdm(content.split('\r\n')):
        word, *origin_affix_list = line.split(' ')
        if word == ''.join(origin_affix_list):
            word_dict[word] = Word(word, origin_affix_list)
        else:
            bad_celex.append(line)
if get_file('CELEX.word.and.root.txt', celex_word_and_root_callback):
    print(f'Load CELEX.word.and.root.txt done [{len(word_dict.keys())} / {len(bad_celex)}]')

## Create Feature for Each Character
Features include:
1. character itself
2. character position
3. is character a vowel
4. distance to the previous breakpoint
5. distance to the next breakpoint
6. how many breakpoints in preceding substring
7. how many breakpoints in succeeding substring
8. word length

In [ ]:
vowels = {'a', 'e', 'i', 'o', 'u'}
def get_ascii(char: str) -> float:
    return (ord(char.lower()) - 110) / 26

def create_char_feature(word: Word) -> list:
    MAX = 100
    text = word.text
    bps = word.get_breakpoints_index() + [MAX]
    features = []
    for idx, character in enumerate(text):
        dis2prev_bp = -1
        dis2next_bp = -1
        prec_bp_count = 0
        succ_bp_count = 0
        for i, bp in enumerate(bps):
            if idx < bp:
                if i > 0:
                    dis2prev_bp = (idx - bps[i-1])
                if bp < MAX:
                    dis2next_bp = bp - idx
                prec_bp_count = i
                succ_bp_count = len(bps) - 1 - i
                break
            if idx == bp:
                dis2prev_bp = 0
                dis2next_bp = 0
                prec_bp_count = i + 1
                succ_bp_count = len(bps) - 2 - i
                break
        features.append([
            1,
            get_ascii(character),
            (idx * 2 / (len(text) - 1)) - 1,
            int(character in vowels),
            dis2prev_bp,
            dis2next_bp,
            prec_bp_count,
            succ_bp_count,
            len(text)
        ])
        # features.append([
        #     'bias',
        #     f'char={character}',
        #     f'vowel={character in vowels}',
        #     f'dis2prev_and_next_bp={dis2prev_bp}:{dis2next_bp}',
        #     f'prec_and_succ_bp_count={prec_bp_count}:{succ_bp_count}',
        # ])
    return features

## Create Train & Test Data

In [ ]:
data_features = [create_char_feature(w) for w in word_dict.values()]
data_label = [w.label for w in word_dict.values()]
train_X, test_X, train_y, test_y = train_test_split(data_features, data_label, test_size=0.2)
train_X = [ee for e in train_X for ee in e]
test_X = [ee for e in test_X for ee in e]
train_y = [ee for e in train_y for ee in e]
test_y = [ee for e in test_y for ee in e]

# train_X = [ee for e in train_y for ee in e]
# train_y = [ee for e in test_y for ee in e]

## Start Train & Test

In [ ]:
forest = ensemble.RandomForestClassifier(n_estimators=100)
forest_fit = forest.fit(train_X, train_y)

test_y_predicted = forest.predict(test_X)

accuracy = accuracy_score(test_y, test_y_predicted)
scores = cross_val_score(forest, train_X + test_X, train_y + test_y, cv=5)
precision, recall, fbeta_score, _ = precision_recall_fscore_support(test_y, test_y_predicted, average='weighted', zero_division=0)
print(accuracy, scores, precision, recall, fbeta_score)

In [ ]:
def decompose(text: str, idx: int, prec_bp_count: int, succ_bp_count: int) -> tuple:
    max_accuracy, max_accuracy_idx = 0, 0
    if len(text) == 1 or prec_bp_count > 10 or succ_bp_count > 10:
        if prec_bp_count > 10:
            print(f'prec ct = {prec_bp_count}')
        if succ_bp_count > 10:
            print(f'succ ct = {succ_bp_count}')
        return text,
    for i in range(0, len(text)):
        t_y = [0] * len(text)
        t_y[i] = 1
        p_y = []
        for j, char in zip(range(0, len(text)), text):
            dis2prev_bp = j if (j < i) else j - i
            dis2next_bp = i - j if (j < i) else (len(text) - j)
            prec_bp_ct = prec_bp_count + (0 if j < i else 1)
            succ_bp_ct = succ_bp_count + (1 if j < i else 0)
#             pd = [1, get_ascii(char), idx + j, int(char in vowels), dis2prev_bp, dis2next_bp, prec_bp_ct, succ_bp_ct, len(text)]
            pd = [1, get_ascii(char), (idx + j) / len(text), int(char in vowels)]
#             print(pd)
            p_y.append(forest.predict([pd]))
        acc = accuracy_score(t_y, p_y)
        print(text, t_y, [list(v) for v in p_y], acc)
        if max_accuracy < acc:
            max_accuracy = acc
            max_accuracy_idx = i
    prec = decompose(text[:max_accuracy_idx], 0, prec_bp_count, succ_bp_count+1) if max_accuracy_idx > 0 else None
    succ = decompose(text[max_accuracy_idx+1:], idx+max_accuracy_idx, prec_bp_count+1, succ_bp_count) if max_accuracy_idx < (len(text) - 1) else None
    decomposition = (prec, succ)
    print(decomposition, max_accuracy)
    if prec and succ:
        return decomposition
    elif prec:
        return (prec, text[max_accuracy_idx])
    elif succ:
        return (text[max_accuracy_idx], succ)
    else:
        return text,
        
# input_text = input()
input_features = decompose('section', 0, 0, 0)
